In [31]:
#import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

#Extract function
def extract(page):
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
    url = f'https://in.indeed.com/jobs?q=Creative%20Technologist&l=Bangalore%2C%20Karnataka&start={page}&vjk=c96f098a92d262e1'
    r = requests.get(url, headers)
    soup = BeautifulSoup(r.content,'html.parser')
    return soup

#Transform function
def transform(soup):
    #Getting list of all job postings
    divs = soup.find_all('div', class_ = 'job_seen_beacon')
    for item in divs:
        title = item.find('h2', class_ ='jobTitle').text.strip()
        location = item.find('div', class_ = 'companyLocation').text.strip()
        company = item.find('span', class_ ='companyName').text.strip()
        date = item.find('span', class_ ='date').text.strip()
        try:
            salary = item.find('div', class_ ='salary-snippet').text.strip()
        except:
            salary = ''
            
        summary = item.find('div',{'class' : 'job-snippet'}).text.strip().replace('\n','')
       
        #Getting full descriptions for each job post
        joburl = "http://indeed.com" + item.h2.a.get('href')
        job_desc_r = requests.get(joburl)
        job_desc_data = job_desc_r.text
        job_desc_soup = BeautifulSoup(job_desc_data, "html.parser")
        job_desc_tag = job_desc_soup.find('div',{'id':'jobDescriptionText'})
        job_description = job_desc_tag.text if job_desc_tag else "N/A"
        
        job = {
            'title': title,
            'company':company,
            'date':date,
            'location':location,
            'salary': salary,
            'summary': summary,
            'description': job_description,
        }
        joblist.append(job)
    return

joblist =[]

#Looping for different pages
for i in range(0,31,10):
    print(f'Getting page, {i}')
    c = extract(i) 
    transform(c)

Getting page, 0
Getting page, 10
Getting page, 20
Getting page, 30


In [33]:
#Creating a dataframe and writing into CSV
df = pd.DataFrame(joblist)
print(df.head())
df.to_csv('CreativeTechnologists.csv')

                              title       company                        date  \
0          Office Manager (On-site)   Tenstorrent  EmployerActive 15 days ago   
1       Process Technologist - SMPO         Shell          Posted30+ days ago   
2  newContent Management Specialist  KnowledgeHut             Posted1 day ago   
3           Software Technologist I       Philips           Posted12 days ago   
4                             Blend        Diageo           Posted14 days ago   

                                location salary  \
0                   Bengaluru, Karnataka          
1                   Bengaluru, Karnataka          
2       HSR Layout, Bengaluru, Karnataka          
3  Hybrid remote in Bengaluru, Karnataka          
4                   Bengaluru, Karnataka          

                                             summary  \
0  Manage Tenstorrent's Covid-19 safety protocols...   
1  Our solutions help our partners grow their bus...   
2  The role will entail creation of we